### Managing State - Agents and Teams

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

from dotenv import load_dotenv

import os

In [ ]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15")
azure_openai_deployment_name = os.getenv(
    "AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-35-turbo")

if not azure_openai_api_key or \
        not azure_openai_endpoint or \
        not azure_openai_api_version or \
        not azure_openai_deployment_name:
    raise ValueError(
        "Please set Azure Open AI Endpoint Details in your environment variables.")

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [ ]:
assistant_agent = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a helpful assistant.",
)

response = await assistant_agent.on_messages(
    [
        TextMessage(
            content="Tell me facts about Dolphins.",
            source="user",
        ),
    ],
    cancellation_token=CancellationToken(),
)

print(response.chat_message.content)

In [ ]:
agent_state = await assistant_agent.save_state()

print(agent_state)

In [ ]:
new_assistant_agent = AssistantAgent(
    name="NewAssistantAgent",
    model_client=model_client,
    system_message="You are a helpful assistant.",
)

await new_assistant_agent.load_state(agent_state)

response = await new_assistant_agent.on_messages(
    [
        TextMessage(
            content="what was the fun facts you shared about Dolphins?",
            source="user",
        ),
    ],
    cancellation_token=CancellationToken(),
)

print(response.chat_message.content)

In [ ]:
assistant_agent = AssistantAgent(
    name="Assistant_Agentv2",
    model_client=model_client,
    system_message="You are a helpful assistant.",
)

agent_team = RoundRobinGroupChat([assistant_agent],
        termination_condition=MaxMessageTermination(2))

stream = agent_team.run_stream(task="Tell me surprising facts about Octopuses.",)

await Console(stream)

team_state = await agent_team.save_state()

In [ ]:
await agent_team.reset()

stream = agent_team.run_stream(task="What was the last animal you talked about?",)

await Console(stream)

In [ ]:
await agent_team.load_state(team_state)

stream = agent_team.run_stream(task="What was the last animal you talked about?",)

await Console(stream)

In [ ]:
import json

with open("team_state.json", "w") as f:
    json.dump(team_state, f, indent=4, sort_keys=True, default=str)